<a href="https://colab.research.google.com/github/MinsooKwak/Data_Analysis/blob/main/LLM/Sparse_vector_vs_Dense_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import gensim.downloader as api
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 비교 예시 문장 2가지
sentences = ["Bill ran from the giraffe toward the dolphin",
             "Bill ran from the dolphin toward the giraffe"]

Sparse Vector

In [4]:
# Sparse Vectorizer
vectorizer = CountVectorizer() # 단어의 등장 빈도 count (unique 단어가 한 차원)

In [5]:
# 예시 문장 fit_transform
sparse_vectors = vectorizer.fit_transform(sentences)  # 7차원

In [6]:
# 결과 출력
print(sparse_vectors.toarray()) # 순서만 바뀌고 sparse vector가 같은 형상

[[1 1 1 1 1 2 1]
 [1 1 1 1 1 2 1]]


Dense Vector

In [7]:
# Dense Vectorizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  # HuggingFace repository에 등록된 사전학습 모델 # 대소문자 구분 없이 학습
model = BertModel.from_pretrained('bert-base-uncased')

# 변환 함수
def create_dense_vector(sentence):
  '''
  * return_tensor = ['pt', 'tf']  # pytorch , tensorflow
  * last_hidden_state 의 평균 :
    문장 단위 embedding을 hidden state의 마지막으로 정의
    문장을 대표하는 표현을 내뱉게 하기 위해 mean(dim=1)
    -> 문장 표현하는 벡터로 사용할 수 있게 됨
  '''
  inputs = tokenizer(sentence, return_tensors="pt") # tokenizing, str -> tensor
  outputs = model(**inputs)
  return outputs.last_hidden_state.mean(dim=1)

# 예시 문장에 대해 Dense Vector 생성
dense_vectors = [create_dense_vector(sentence) for sentence in sentences]

# 결과 출력
for vector in dense_vectors:
  print(vector)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tensor([[-2.3714e-02,  5.7633e-02, -1.8486e-01,  1.9893e-01,  1.8176e-01,
         -1.4805e-01, -3.2456e-01,  4.0638e-01, -2.7499e-01, -2.3101e-01,
         -1.1523e-01, -1.1990e-02,  4.6302e-04, -1.2367e-01, -1.0451e-01,
          1.0718e-01, -1.5655e-02, -1.4022e-01, -2.1803e-01, -1.0421e-01,
         -3.4618e-02, -2.3747e-01, -3.4193e-01,  9.7915e-02,  3.4513e-01,
         -2.5600e-01,  2.8723e-02,  2.8422e-01, -1.1099e-01,  1.2839e-01,
          3.2615e-01, -1.2789e-01,  4.8784e-02,  3.5963e-02, -2.8813e-01,
          1.1239e-01,  1.0251e-01, -2.7317e-01, -4.4257e-01,  1.3002e-01,
         -1.6435e-01, -1.2559e-01,  1.6004e-01,  1.0129e-01,  1.8999e-01,
         -5.1440e-02,  3.8157e-01,  3.6777e-03,  3.3611e-01,  4.7525e-02,
         -5.0798e-01,  4.4805e-01, -2.0733e-01,  9.3377e-02,  4.8761e-02,
          3.5601e-01,  2.2678e-01, -9.8933e-02, -4.6186e-01, -3.7770e-01,
          1.7621e-01,  1.2030e-01,  3.4253e-01, -1.8605e-01, -1.4936e-01,
          2.7348e-01,  5.6670e-02,  4.

In [8]:
# 결과 비교
print("Sparse Vector:")
print(sparse_vectors.toarray())

print("\nDense Vector:")
for vector in dense_vectors:
  print(vector)

Sparse Vector:
[[1 1 1 1 1 2 1]
 [1 1 1 1 1 2 1]]

Dense Vector:
tensor([[-2.3714e-02,  5.7633e-02, -1.8486e-01,  1.9893e-01,  1.8176e-01,
         -1.4805e-01, -3.2456e-01,  4.0638e-01, -2.7499e-01, -2.3101e-01,
         -1.1523e-01, -1.1990e-02,  4.6302e-04, -1.2367e-01, -1.0451e-01,
          1.0718e-01, -1.5655e-02, -1.4022e-01, -2.1803e-01, -1.0421e-01,
         -3.4618e-02, -2.3747e-01, -3.4193e-01,  9.7915e-02,  3.4513e-01,
         -2.5600e-01,  2.8723e-02,  2.8422e-01, -1.1099e-01,  1.2839e-01,
          3.2615e-01, -1.2789e-01,  4.8784e-02,  3.5963e-02, -2.8813e-01,
          1.1239e-01,  1.0251e-01, -2.7317e-01, -4.4257e-01,  1.3002e-01,
         -1.6435e-01, -1.2559e-01,  1.6004e-01,  1.0129e-01,  1.8999e-01,
         -5.1440e-02,  3.8157e-01,  3.6777e-03,  3.3611e-01,  4.7525e-02,
         -5.0798e-01,  4.4805e-01, -2.0733e-01,  9.3377e-02,  4.8761e-02,
          3.5601e-01,  2.2678e-01, -9.8933e-02, -4.6186e-01, -3.7770e-01,
          1.7621e-01,  1.2030e-01,  3.4253e-01,

In [9]:
# 코사인 유사도 계산 (Sparse Vector 기준)
sparse_similarity = cosine_similarity(sparse_vectors[0], sparse_vectors[1])
print("코사인 유사도 (Sparse):", sparse_similarity[0][0])

코사인 유사도 (Sparse): 1.0


완벽하게 똑같다고 봄

In [10]:
# 코사인 유사도 계산 (Dense Vector 기준)
def calculate_cosine_similarity(vector1, vector2):
  return F.cosine_similarity(vector1, vector2).item()

dense_similarity = calculate_cosine_similarity(dense_vectors[0], dense_vectors[1])
print("Cosine Similarity (Dense Vectors):", dense_similarity)

Cosine Similarity (Dense Vectors): 0.9742187261581421


거의 비슷하지만 아주 미세한 의미적인 차이가 있다